In the cells below, set `log_path` to a valid path and your choice of filename. The results of your lasso operations will be saved here.

Set `results_dir` and the path to the `grid_files` in the next line to match the dates for the case you're analyzing. the wildcard match to flash_extent and the nc_field below it are set to look at the 2D flash extent density grids, which is probably what you want.

To use the tool, run all the cells. An interactive lasso widget that pops up. Use the frame slider to scroll through each time in the gridded data. Ckick the "Draw Lasso" button to practice drawing a lasso around a storm of interest. 

Once you have practiced, check the box to log lassos. When checked, any lassos drawn will be saved to the lasso log file.

Recommended best practice is to start at the first frame, define a cell, advance to the next frame, and then define another cell. If you skip a few frames, that's ok; in the analysis code that uses the lasso log file, the first lasso you defined persists until a new one is available.

You can watch the lasso log file fill up by doing `tail -f lasso_log_filename` in a separate terminal window.

When you're done, just quit the notebook.

In [1]:
import os, glob
import tables, pandas
import numpy as np

import warnings
# warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning, module='.*/IPython/.*')

In [2]:
%matplotlib qt4
import matplotlib.pyplot as plt

/Users/ebruning/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
log_path = './DemoNCGridLassoLog.txt'

In [4]:
from lmatools.grid.grid_collection import LMAgridFileCollection
    
results_dir = "flashsort"
nc_filenames=glob.glob(os.path.join(results_dir, "grid_files/2014/May/26/*_flash_extent.nc"))
nc_field = 'flash_extent'

# print nc_filenames
NCs = LMAgridFileCollection(nc_filenames, nc_field, x_name='longitude', y_name='latitude')
geosys, mapproj = NCs.get_projection()

projection not found, assuming lat, lon grid


/Users/ebruning/anaconda/lib/python2.7/site-packages/scipy/io/netcdf.py:297: RuntimeWarning: Cannot close a netcdf_file opened with mmap=True, when netcdf_variables or arrays referring to its data still exist. All data arrays obtained from such files refer directly to data on disk, and must be copied before the file can be cleanly closed. (See netcdf_file docstring for more information on mmap.)
  ), category=RuntimeWarning)


In [5]:
# This cell is not necessary for the lasso analysis, but shows how
# one might overlay additional data on the plot in order to guide 
# the lasso analyses - in this case, which cell was being sampled by
# two radars scanning in RHI mode?

ka1_loc = -102.13410, 33.30546, 1004.0
ka2_loc = -102.09339, 33.40950, 1018.0
ka1_az = 180. + 163. - 360.0
ka2_az = 0. + 249.
radar_dist = 20.0e3

ka1_dx = radar_dist * np.cos(np.deg2rad(90.0-ka1_az))
ka1_dy = radar_dist * np.sin(np.deg2rad(90.0-ka1_az))
ka2_dx = radar_dist * np.cos(np.deg2rad(90.0-ka2_az))
ka2_dy = radar_dist * np.sin(np.deg2rad(90.0-ka2_az))

ka1_xyz = mapproj.fromECEF(*geosys.toECEF(*ka1_loc))
ka2_xyz = mapproj.fromECEF(*geosys.toECEF(*ka2_loc))

ka1_radial = [[ka1_xyz[0], ka1_xyz[0]+ka1_dx], 
              [ka1_xyz[1], ka1_xyz[1]+ka1_dy]
             ]
ka2_radial = [[ka2_xyz[0], ka2_xyz[0]+ka2_dx], 
              [ka2_xyz[1], ka2_xyz[1]+ka2_dy]
             ]

def plot_ka(lasso_widget, ax):
    ax.plot(ka1_radial[0], ka1_radial[1], 'k')
    ax.annotate('Ka1', xy=(ka1_radial[0][0], ka1_radial[1][0]))
    ax.plot(ka2_radial[0], ka2_radial[1], 'k')
    ax.annotate('Ka2', xy=(ka2_radial[0][0], ka2_radial[1][0]))
    ax.axis((-75e3, 0e3, -75e3, 0e3))
    ax.tick_params(axis='both', labelsize=12)


## Interactive lasso widget

In [6]:
# from ipywidgets import interactive
from IPython.display import display

In [7]:
from GridLassoTools import NCgridLasso, NCgridLassoWidgets
nclasso = NCgridLasso(NCs, log_path=log_path)
widget = NCgridLassoWidgets(nclasso) #, overplot_func = plot_ka)
display(widget.container)

INFO:GridLassoTools.Poly:{"field": "flash_extent", "lon_verts": [-103.13709677419355, -103.14516129032259, -101.24193548387098, -101.33064516129033, -103.16935483870968, -103.13709677419355], "x_verts": [-103.13709677419355, -103.14516129032259, -101.24193548387098, -101.33064516129033, -103.16935483870968, -103.13709677419355], "title": "Lasso Verts", "y_verts": [34.4375, 33.285714285714285, 33.339285714285715, 34.982142857142854, 35.05357142857143, 34.4375], "frame_time": "2014-05-26T09:40:00", "filename": "flashsort/grid_files/2014/May/26/WTLMA_20140526_094000_600_10src_0.0108deg-dx_flash_extent.nc", "frame_id": 0, "lat_verts": [34.437499999999986, 33.285714285714285, 33.33928571428572, 34.98214285714286, 35.05357142857144, 34.437499999999986]}


releasing lock
projection not found, assuming lat, lon grid


INFO:GridLassoTools.Poly:{"field": "flash_extent", "lon_verts": [-103.25, -103.25, -100.70161290322581, -101.16129032258065, -102.74193548387098, -103.25], "x_verts": [-103.25, -103.25, -100.70161290322581, -101.16129032258065, -102.74193548387098, -103.25], "title": "Lasso Verts", "y_verts": [34.61607142857143, 33.05357142857143, 33.19642857142857, 35.25, 35.232142857142854, 34.61607142857143], "frame_time": "2014-05-26T09:50:00", "filename": "flashsort/grid_files/2014/May/26/WTLMA_20140526_095000_600_10src_0.0108deg-dx_flash_extent.nc", "frame_id": 10, "lat_verts": [34.61607142857142, 33.05357142857142, 33.19642857142857, 35.25, 35.23214285714284, 34.61607142857142]}


releasing lock
projection not found, assuming lat, lon grid
